# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([105])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])
    ArbitraryUnitary(params[15:30], wires=[2, 3])
    ArbitraryUnitary(params[30:45], wires=[4, 5])
    ArbitraryUnitary(params[45:60], wires=[6, 7])

    # Second layer
    ArbitraryUnitary(params[60:75], wires=[1, 2])
    ArbitraryUnitary(params[75:90], wires=[5, 6])

    # Third layer
    ArbitraryUnitary(params[90:105], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5018007202881152
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6379351740696279
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6696278511404562
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6672268907563026
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7104441776710684
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7015606242496999
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7190876350540216
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.724609843937575
best accuracy so far!
epoch 71

epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7339735894357743
test_accuracies: [0.7190876350540216, 0.724609843937575, 0.7202881152460985, 0.7313325330132052, 0.7171668667466986, 0.7178871548619448, 0.7260504201680672, 0.7056422569027612, 0.7166866746698679, 0.724609843937575, 0.724609843937575, 0.7313325330132052, 0.7270108043217287, 0.7162064825930372, 0.7404561824729892, 0.7260504201680672, 0.7198079231692678, 0.7222088835534214, 0.723889555822329, 0.7339735894357743, 0.7366146458583434, 0.726530612244898, 0.7368547418967587, 0.7109243697478991, 0.726530612244898, 0.7272509003601441, 0.7255702280912365, 0.7123649459783914, 0.7130852340936374, 0.7339735894357743]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7435774309723889
best accuracy so far!
test_accuracies: [0.724609843937575, 0.7202881152460985, 0.7313325330132052, 0.7171668667466986, 0.7178871548619448, 0.7260504201680672, 0.705642256

epoch 458
epoch 459
epoch 460
accuracy: 0.7286914765906363
test_accuracies: [0.7313325330132052, 0.7270108043217287, 0.7162064825930372, 0.7404561824729892, 0.7260504201680672, 0.7198079231692678, 0.7222088835534214, 0.723889555822329, 0.7339735894357743, 0.7366146458583434, 0.726530612244898, 0.7368547418967587, 0.7109243697478991, 0.726530612244898, 0.7272509003601441, 0.7255702280912365, 0.7123649459783914, 0.7130852340936374, 0.7339735894357743, 0.7435774309723889, 0.7358943577430972, 0.7339735894357743, 0.723889555822329, 0.7260504201680672, 0.720048019207683, 0.7351740696278511, 0.7169267707082834, 0.7274909963985594, 0.7145258103241297, 0.7286914765906363]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7294117647058824
test_accuracies: [0.7270108043217287, 0.7162064825930372, 0.7404561824729892, 0.7260504201680672, 0.7198079231692678, 0.7222088835534214, 0.723889555822329, 0.7339735894357743, 0.7366146458583434, 0.

epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7205282112845138
test_accuracies: [0.7109243697478991, 0.726530612244898, 0.7272509003601441, 0.7255702280912365, 0.7123649459783914, 0.7130852340936374, 0.7339735894357743, 0.7435774309723889, 0.7358943577430972, 0.7339735894357743, 0.723889555822329, 0.7260504201680672, 0.720048019207683, 0.7351740696278511, 0.7169267707082834, 0.7274909963985594, 0.7145258103241297, 0.7286914765906363, 0.7294117647058824, 0.7284513805522208, 0.7135654261704683, 0.721968787515006, 0.7253301320528212, 0.7092436974789915, 0.7015606242496999, 0.7284513805522208, 0.7099639855942377, 0.7042016806722688, 0.7068427370948379, 0.7205282112845138]
epoch 581
epoch 582
epoch 583
epoch 584
epoch 585
epoch 586
epoch 587
epoch 588
epoch 589
epoch 590
accuracy: 0.7267707082833132
test_accuracies: [0.726530612244898, 0.7272509003601441, 0.7255702280912365, 0.7123649459783914, 0.7130852340936374, 0.733973589

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 1.33782768  1.80253108  0.1185268   0.67735127  1.44879974  0.1998976
  0.00222416  0.60421382  0.73314355  0.45567858  0.91466741 -0.89444963
 -0.06270789  0.5844192   0.66930422  0.29601331  1.53402102 -0.11761829
  1.20380507  1.26433615  1.35627801  0.31275712  0.58740638 -0.61553864
  0.22152669  0.26666141  1.23045307  0.8111025   0.73290955  0.23132314
 -0.4079694   0.42261686  0.62233373  0.30545075  0.41571281 -0.15494426
  1.39523059  0.51471465  1.33866825 -0.03538094  0.59184583  0.92012998
  0.28265405  0.32509867  0.51314281  0.08286415  0.60703232  0.89366819
  0.54502988  1.23321934  1.43967586  0.82064463  0.88068045  0.48090707
  0.29641629  0.55871548  0.35566708  0.52954201  0.60903401  0.76905057
  0.16196136  1.06091665  0.87731956  0.55560348 -0.21309756  0.28719582
  0.78248708  1.60023672  0.41185613  0.43562988  0.45713996  0.12124175
  0.68840753  0.98551898  0.67369851  0.98064638  0.44655896  1.10057224
  0.11941049  0.6184419  

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7431428571428571


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 9.845602989196777
training time: 17094.262204885483
test time: 541.5670757293701
total time: 17645.67488360405
